In [9]:
import pandas as pd

# Giả sử bạn đã load DataFrame từ file hoặc từ list of dicts:
df = pd.read_json("failed_data.json")

# Lọc và in
no_resp_indices = df.loc[
    df["status"] == "parse_failed",
    "global_index"
].tolist()

print(no_resp_indices)


[2198, 2253, 2260, 2273, 2275, 2290, 2291, 2299, 2339, 2367, 2374, 2375, 2410, 2422, 2424, 2434, 2457, 2463, 2465, 2466, 2472, 2483, 2484, 2486, 2509, 2524, 2527, 2530, 2570, 2613, 2619, 2624, 2634, 2635, 2638, 2643, 2646, 2647, 2725, 2728, 2729, 2733, 2742, 2755, 2780, 2781]


In [41]:
len(no_resp_indices)

52

In [10]:
import json
import re

def parse_symptoms_raw(raw: str):
    if isinstance(raw, list):
        return raw
    if raw is None:
        return []

    raw = raw.strip().strip("```json").strip("```").strip()

    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        pass

    # Regex đơn cấp, phù hợp với re module
    json_objects = re.findall(r'\{[^{}]*\}', raw)
    parsed = []
    for obj in json_objects:
        try:
            parsed.append(json.loads(obj))
        except json.JSONDecodeError:
            continue
    return parsed

def fix_json_entry(entry):
    raw = entry.get("symptoms_raw")
    parsed = parse_symptoms_raw(raw)

    if parsed:
        entry["symptoms_raw"] = parsed
        entry["status"] = "ok"
    else:
        entry["symptoms_raw"] = []
        entry["status"] = "parse_failed"
    return entry

# ==== ⚙️ Cấu hình: chỉnh các index cần sửa ====
target_indexes = no_resp_indices  # 🟡 Thay bằng global_index bạn cần sửa
input_path = "final_test.json"
output_path = "fixed_selected.json"

# ==== 📦 Load file JSON ====
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ==== 🛠 Sửa những entry có global_index trong danh sách ====
fixed_data = []
for entry in data:
    if entry.get("global_index") in target_indexes:
        fixed_data.append(fix_json_entry(entry))
    else:
        fixed_data.append(entry)

# ==== 💾 Ghi ra file mới ====
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(fixed_data, f, indent=2, ensure_ascii=False)

print(f"✅ Đã sửa xong {len(target_indexes)} mục và lưu vào {output_path}")


✅ Đã sửa xong 46 mục và lưu vào fixed_selected.json


In [2]:
import json

def replace_entries_by_index(file1_path, file2_path, index_list, output_path):
    """
    Thay thế các entry trong file1 theo global_index nếu nằm trong index_list,
    sử dụng dữ liệu từ file2.
    """
    with open(file1_path, 'r', encoding='utf-8') as f1:
        data1 = json.load(f1)
    with open(file2_path, 'r', encoding='utf-8') as f2:
        data2 = json.load(f2)

    # Tạo lookup từ file2 theo global_index
    lookup2 = {entry['global_index']: entry for entry in data2}

    # Duyệt qua file1, thay thế nếu index nằm trong danh sách
    new_data = []
    for entry in data1:
        idx = entry.get('global_index')
        if idx in index_list and idx in lookup2:
            new_data.append(lookup2[idx])  # thay bằng bản từ file2
        else:
            new_data.append(entry)  # giữ nguyên

    # Ghi ra file mới
    with open(output_path, 'w', encoding='utf-8') as fout:
        json.dump(new_data, fout, ensure_ascii=False, indent=2)

# Ví dụ sử dụng
file1 = 'final_test.json'
file2 = 'failed_data.json'
index_list = no_resp_indices  # danh sách global_index cần thay thế
output_file = 'final_test.json'

replace_entries_by_index(file1, file2, index_list, output_file)
